# Book Recommendation

Data:
https://www.kaggle.com/code/fahadmehfoooz/book-recommendation-system/input

Goal:
- recommend the top k books to a user
- use another libray as scikit-surprise because it is unmaintained and doesnt work anymore for python 3.12
- use one algorithm where i can use HPO

## Imports

In [39]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

### Data Overview

In [40]:
books_df = pd.read_csv("./data/books/Books.csv").set_index("ISBN")
books_df.head()

C:\Users\Eric User\AppData\Local\Temp\ipykernel_3540\2748821274.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("./data/books/Books.csv").set_index("ISBN")


,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,
0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg
0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg
0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg
0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg


In [15]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271360 entries, 0195153448 to 0767409752
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Book-Title           271360 non-null  object
 1   Book-Author          271358 non-null  object
 2   Year-Of-Publication  271360 non-null  object
 3   Publisher            271358 non-null  object
 4   Image-URL-S          271360 non-null  object
 5   Image-URL-M          271360 non-null  object
 6   Image-URL-L          271357 non-null  object
dtypes: object(7)
memory usage: 16.6+ MB


Why is Year-Of-Publication not numerical?

In [29]:
books_df[~books_df['Year-Of-Publication'].apply(lambda x : str(x).isdigit())]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
209538,078946697X,"DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)\"";Michael Teitelbaum""",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/078946697X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/078946697X.01.LZZZZZZZ.jpg,NaN
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-Marie Gustave Le ClÃ?Â©zio""",2003,Gallimard,http://images.amazon.com/images/P/2070426769.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/2070426769.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/2070426769.01.LZZZZZZZ.jpg,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)\"";James Buckley""",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0789466953.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0789466953.01.LZZZZZZZ.jpg,NaN


In [4]:
books_df.describe()

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271358,271360,271358,271360,271360,271357
unique,242135,102022,202,16807,271044,271044,271041
top,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,27,632,13903,7535,2,2,2


In [6]:
users_df = pd.read_csv("./data/books/Users.csv").set_index("User-ID")
users_df.head()

,Location,Age
User-ID,,
1,"nyc, new york, usa",NaN
2,"stockton, california, usa",18.0
3,"moscow, yukon territory, russia",NaN
4,"porto, v.n.gaia, portugal",17.0
5,"farnborough, hants, united kingdom",NaN


In [10]:
users_df.describe()

,Age
count,168096.000000
mean,34.751434
std,14.428097
min,0.000000
25%,24.000000
50%,32.000000
75%,44.000000
max,244.000000


Users with Age 0?

In [35]:
users_df[users_df['Age'] < 12]

,Location,Age
User-ID,,
220,"bogota, bogota, colombia",0.0
470,"indianapolis, indiana, usa",0.0
562,"adfdaf, australian capital territory, albania",0.0
613,"ankara, n/a, turkey",1.0
671,"jeddah, jeddah, saudi arabia",1.0
...,...,...
277405,"bismarck, north dakota, usa",10.0
277909,"phoenix, arizona, usa",2.0
278266,"matale, sri lanka, sri lanka",7.0


In [9]:
users_df.describe(include="object")

,Location
count,278858
unique,57339
top,"london, england, united kingdom"
freq,2506


In [17]:
ratings_df = pd.read_csv("./data/books/Ratings.csv", dtype={"User-ID" : str}).sample(50000) 
# pivoting everything above 40000 kills my RAM >10GB -> for a real world example calculate for different chunks
ratings_df.head()

,User-ID,ISBN,Book-Rating
646941,156269,0451198719,0
765767,185233,0446679593,0
376466,90694,3927568074,9
318673,76352,055327659X,0
937554,227447,0140149902,0


In [18]:
ratings_df.describe()

,Book-Rating
count,50000.00000
mean,2.88106
std,3.85588
min,0.00000
25%,0.00000
50%,0.00000
75%,7.00000
max,10.00000


In [7]:
print("Books Shape: " ,books_df.shape )
print("Users Shape: " ,users_df.shape )
print("Ratings Shape: " ,ratings_df.shape )
print("\nNull values in DFs:")
print(f"Books:\n{books_df.isnull().sum()}\n")
print(f"Users:\n{users_df.isnull().sum()}\n")
print(f"Ratings:\n{ratings_df.isnull().sum()}")

Books Shape:  (271360, 7)
Users Shape:  (278858, 2)
Ratings Shape:  (50000, 3)

Null values in DFs:
Books:
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

Users:
Location         0
Age         110762
dtype: int64

Ratings:
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


### Inferences:
- Book Dataframe
  - we know that the **Year-Of-Publication** should be of numerical type; however, the year data contains non-numeric values
  - it's evident that the CSV file is corrupted at three rows, which results in moved columns

- Users Dataframe
  - there are over 1200 users with ages less than ten, users with age zero, and users with no specified age
  - the majority of our users are in their 30s and from England

- Ratings Dataframe
  - contains ratings between 0 and 10
  - as given in our dataset description, users can rate explicitly from 1 to 10 or implicitly with a rating of 0, indicating that a user purchased the book but didn't rate it

## Data Cleaning and Transformations

Clean Book Data

In [186]:
# clean corrupted rows
corrupt_df = books_df[~books_df['Year-Of-Publication'].apply(lambda x : str(x).isdigit())]
clean_book_df = pd.DataFrame(list(corrupt_df['Book-Title'].str.replace('\\', '').str.replace('"', '').str.split(";").values), columns=['Book-Title', 'Book-Author'])
corrupt_df = corrupt_df.shift(axis=1)
corrupt_df['Book-Title'] = clean_book_df['Book-Title'].values
corrupt_df['Book-Author'] = clean_book_df['Book-Author'].values
books_df.update(corrupt_df)
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype(int)

In [177]:
corrupt_df['Book-Title'] = clean_book_df['Book-Title'].values
corrupt_df

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,
078946697X,"DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)","DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)\"";Michael Teitelbaum""",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/078946697X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/078946697X.01.LZZZZZZZ.jpg
2070426769,"Peuple du ciel, suivi de 'Les Bergers","Peuple du ciel, suivi de 'Les Bergers\"";Jean-Marie Gustave Le ClÃ?Â©zio""",2003,Gallimard,http://images.amazon.com/images/P/2070426769.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/2070426769.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/2070426769.01.LZZZZZZZ.jpg
0789466953,"DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)","DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)\"";James Buckley""",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0789466953.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0789466953.01.LZZZZZZZ.jpg


In [166]:
clean_book_df[['Book-Title', 'Book-Author']]

,Book-Title,Book-Author
0,"DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)",Michael Teitelbaum
1,"Peuple du ciel, suivi de 'Les Bergers",Jean-Marie Gustave Le ClÃ?Â©zio
2,"DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)",James Buckley


In [168]:

corrupt_df.loc[corrupt_df.index.isin(clean_book_df.index), ['Book-Title', 'Book-Author']] =

,Book-Title,Book-Author
0,078946697X,"DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)\"";Michael Teitelbaum"""
1,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-Marie Gustave Le ClÃ?Â©zio"""
2,0789466953,"DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)\"";James Buckley"""


In [162]:
corrupt_df = corrupt_df[['Book-Title', 'Book-Author']].update(clean_book_df[['Book-Title', 'Book-Author']])
corrupt_df

In [159]:
print(corrupt_df)

None


In [130]:
clean_book_df['Book-Title']

0                DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)
1                                                         Peuple du ciel, suivi de 'Les Bergers
2    DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)
Name: Book-Title, dtype: object

In [44]:

corrupt_df['Book Title'], corrupt_df['Author'] = corrupt_df['Book Title'].str.split(';Name', 1).str
corrupt_df

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
ISBN,,,,,,,
078946697X,None,"DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)\"";Michael Teitelbaum""",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/078946697X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/078946697X.01.LZZZZZZZ.jpg
2070426769,None,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-Marie Gustave Le ClÃ?Â©zio""",2003,Gallimard,http://images.amazon.com/images/P/2070426769.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/2070426769.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/2070426769.01.LZZZZZZZ.jpg
0789466953,None,"DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)\"";James Buckley""",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0789466953.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0789466953.01.LZZZZZZZ.jpg


In [19]:
df = books_df.merge(ratings_df, on = 'ISBN')
df = df.dropna() # we can drop null values because we cant use every rating [memory restrictions]
df = df.drop(columns=["Image-URL-S","Image-URL-M", "Image-URL-L"], axis=1)
df["Book-Title"]= df["Book-Title"].apply(lambda x: x.strip())
df["Publisher"]= df["Publisher"].apply(lambda x: x.strip())

In [20]:
df["Book-Title"]

0                                   The Kitchen God's Wife
1                                   The Kitchen God's Wife
2                                   The Kitchen God's Wife
3                                            The Testament
4                                            The Testament
                               ...                        
44930         Llewellyn's Witchy Day Planner 2004 Calendar
44931    Lily's Church Camp Adventure (Young Women of F...
44932                        Cross My Heart (Hidden Diary)
44933    True Stories: Girls' Inspiring Stories of Cour...
44934                                      Singularity Sky
Name: Book-Title, Length: 44935, dtype: object

In [21]:
user_item_pivot_matrix_df = df.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating', fill_value=0)
user_item_pivot_matrix_df.head()

ISBN,0000913154,0002116286,0002157950,0002212080,0002215497,0002215810,0002226162,000222674X,0002227150,0002231352,...,9963842801,9977671281,9997404580,9997407210,9997412044,B000051WXP,B00005NCS7,B00008WFXL,B0000E63CJ,B0001GDNCK
User-ID,,,,,,,,,,,,,,,,,,,,,
100009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 10 # hyperparameter

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(user_item_pivot_matrix_df.values, k = NUMBER_OF_FACTORS_MF)

In [27]:
sigma = np.diag(sigma)
sigma.shape

(10, 10)

In [28]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 3.29339241e-20,  4.26254609e-35,  1.76552173e-35, ...,
         0.00000000e+00,  9.40700423e-37,  0.00000000e+00],
       [-1.14616909e-18,  3.49933776e-32, -2.97165609e-34, ...,
         0.00000000e+00, -1.05429315e-35,  0.00000000e+00],
       [-4.42495644e-12,  3.58985471e-24,  8.03510865e-27, ...,
         0.00000000e+00,  1.87163588e-26,  0.00000000e+00],
       ...,
       [-5.02221942e-18, -7.98125787e-35,  1.06974486e-35, ...,
         0.00000000e+00, -2.82890081e-36,  0.00000000e+00],
       [ 7.35812512e-18,  4.17852759e-33, -1.00200285e-34, ...,
         0.00000000e+00,  6.73468040e-35,  0.00000000e+00],
       [-1.35256710e-07,  1.95327995e-21,  2.23003309e-24, ...,
         0.00000000e+00,  1.34226189e-23,  0.00000000e+00]])

In [39]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.ISBN == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.ISBN== id]['Book-Title'].values[0])

In [40]:
k = 50
isbn = "0399135782"
top_n = 3
sliced = Vt.T[:, :k] # representative data
# top_cosine_sim funktioniert gerade nicht, weil ich davor mit ISBNs arbeite
# aber das selbst programmieren zu müssen scheint eh nicht der beste Weg zu sein, lieber sklearn sachen angucken, die scheinen dort besser zu passen
similar_books(df, isbn, top_cosine_similarity(sliced, isbn, top_n))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [43]:
Vt.T[:, :k]

array([[-1.59242852e-01,  2.08735090e-04, -2.56241076e-06, ...,
        -9.43426767e-08,  3.60954048e-07,  2.19971557e-04],
       [-1.35929624e-18,  2.44757016e-22, -4.01050543e-17, ...,
         5.33994705e-18, -1.32136717e-19,  2.05405342e-19],
       [ 0.00000000e+00,  1.87048683e-34, -1.06000628e-34, ...,
        -1.06757915e-18, -1.86347248e-20, -2.11758237e-22],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.82718598e-20,  2.85037209e-20,  1.33643192e-20, ...,
         1.78793860e-20,  1.13710968e-20, -1.74303675e-20],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [35]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating
0,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,180297,0
1,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,136509,0
2,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,8,0
3,0440234743,The Testament,John Grisham,1999,Dell,213733,0
4,0440234743,The Testament,John Grisham,1999,Dell,273979,9
...,...,...,...,...,...,...,...
44930,0738702315,Llewellyn's Witchy Day Planner 2004 Calendar,Ellen Dugan,2003,Llewellyn Publications,269009,4
44931,031070264X,Lily's Church Camp Adventure (Young Women of F...,Nancy N. Rue,2003,Zonderkidz,269557,8
44932,0764224808,Cross My Heart (Hidden Diary),Sandra Byrd,2001,Bethany House Publishers,269557,10
44933,1584857447,True Stories: Girls' Inspiring Stories of Cour...,Trula Magruder,2003,American Girl,269557,10
